In [74]:
import pandas as pd
import geopandas as geo
import numpy as np
import matplotlib.pyplot as plt
from shapely import wkt


In [75]:
Festnetz = geo.read_file("AUTmap_Merged_festnetz.gpkg")

In [76]:
Festnetz_gov = geo.read_file("festnetz_gov.gpkg")

In [77]:
Festnetz = Festnetz2.drop_duplicates("ID")

In [78]:
Festnetz_gov["Download_Max"] = Festnetz_gov.groupby("ID")["download"].transform("max")

In [79]:
Festnetz_gov["Download_Min"] = Festnetz_gov.groupby("ID")["download"].transform("min")

In [80]:
Festnetz_gov = Festnetz_gov.drop_duplicates("ID")

In [81]:
Festnetz_gov_merged = pd.merge(Festnetz_gov,Festnetz,on="ID", how="left")

In [82]:
Festnetz_gov_merged["Download_Mean"] = Festnetz_gov_merged["Download_Mean"].fillna(0)

In [83]:
Festnetz_gov_merged.columns

Index(['ID', 'NAME_x', 'l000100v3', 'infrastrukturanbieterin', 'technik',
       'download', 'upload', 'bearbeitung_bbb', 'geometry_x', 'Download_Max_x',
       'Download_Min', 'NAME_y', 'index_right', 'network_type',
       'download_kbit', 'upload_kbit', 'Download_Mean', 'Download_Max_y',
       'Download_Median', 'Download_Count', 'geometry_y'],
      dtype='object')

In [84]:
Festnetz_gov_merged = Festnetz_gov_merged.drop(["bearbeitung_bbb","geometry_y"],axis=1)

In [85]:
Festnetz_gov_merged["Download_Mean"] = Festnetz_gov_merged["Download_Mean"].fillna(0)

In [86]:
Festnetz_gov_merged["Download_Max_y"] = Festnetz_gov_merged["Download_Max_y"].fillna(0)

In [87]:
Festnetz_gov_merged["Daten_wurden_verglichen"] = 0

In [88]:
Festnetz_gov_merged["relativerFehler_Max/Mean"] = (1000*Festnetz_gov_merged["Download_Max_x"] -Festnetz_gov_merged["Download_Mean"] )/(1000*Festnetz_gov_merged["Download_Max_x"])

In [89]:
Festnetz_gov_merged["relativerFehler_Max/Max"] = (1000*Festnetz_gov_merged["Download_Max_x"] -Festnetz_gov_merged["Download_Max_y"] )/(1000*Festnetz_gov_merged["Download_Max_x"])

In [90]:
Festnetz_gov_merged["relativerFehler_Min/Mean"] = (1000*Festnetz_gov_merged["Download_Min"] -Festnetz_gov_merged["Download_Mean"] )/(1000*Festnetz_gov_merged["Download_Min"])

In [91]:
Festnetz_gov_merged["relativerFehler_Min/Max"] = (1000*Festnetz_gov_merged["Download_Min"] -Festnetz_gov_merged["Download_Max_y"] )/(1000*Festnetz_gov_merged["Download_Min"])

In [92]:
Festnetz_gov_merged.loc[(Festnetz_gov_merged['download'] > 1)
                        &(Festnetz_gov_merged['Download_Mean'] > 1), 'Daten_wurden_verglichen'] = 1

In [93]:
Festnetz_gov_merged_mitVergleichen = Festnetz_gov_merged[Festnetz_gov_merged["Daten_wurden_verglichen"] != 0]

In [96]:
Festnetz_gov_merged_mitVergleichen_Mean = Festnetz_gov_merged_mitVergleichen[Festnetz_gov_merged["relativerFehler_Max/Mean"] > -5]

C:\Users\Paul\AppData\Local\Temp\ipykernel_3940\4190153981.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  Festnetz_gov_merged_mitVergleichen_Mean = Festnetz_gov_merged_mitVergleichen[Festnetz_gov_merged["relativerFehler_Max/Mean"] > -5]


In [97]:
gdf2 = geo.GeoDataFrame(
    Festnetz_gov_merged_mitVergleichen_Mean, geometry = Festnetz_gov_merged_mitVergleichen_Mean.geometry_x,crs="EPSG:31287" )

In [98]:
Grenze = pd.read_csv("Nö_test2.csv")

In [99]:
geometry = Grenze["WKT"].apply(wkt.loads)
Grenze["geometry"] = Grenze["WKT"].apply(wkt.loads)
Grenze = geo.GeoDataFrame(Grenze,geometry =geometry , crs="EPSG:31287")

In [100]:
gdf2=gdf2.drop(labels=['index_right'], axis=1)

In [101]:
Nö_gdf = geo.sjoin(gdf2, Grenze,how="inner",op="intersects")

In [102]:
Nö_gdf = Nö_gdf.drop( ['index_right', 'WKT', 'id'], axis=1)

In [105]:
Nö_gdf = Nö_gdf.drop( ['geometry_x'], axis=1)

In [106]:
Nö_gdf.to_file('Festnetz_rel_Errors_NÖ.gpkg', driver='GPKG')  

C:\Users\Paul\anaconda3\envs\NewForGeopandas\lib\site-packages\geopandas\io\file.py:299: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
